In [1]:
import pandas as pd
import numpy as np
from keras.layers import Activation, Dense, Dropout, SpatialDropout1D,Input,Masking,Bidirectional, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.models import Sequential, Model
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from random import seed
#seed(1)
import tensorflow as tf
from keras.layers import Input, Dense
from keras.models import Model

#defining the early stopping criteria
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,restore_best_weights=True,patience=3)


Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [3]:
gs = pd.read_csv('LSTM_1_hour.csv')
gs.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
cols_to_use = ['uhid','pulserate', 'ecg_resprate',
       'spo2', 'heartrate', 'mean_bp', 'sys_bp', 'dia_bp',
       'peep', 'pip', 'map', 'tidalvol', 'minvol', 'ti', 'fio2',
       'abd_difference_y',
       'abdomen_girth_y','currentdateheight',
       'currentdateweight','dischargestatus', 
       'new_ph', 
       'rbs',  'stool_day_total', 
       'temp', 'total_intake', 'totalparenteralvolume',
       'tpn-tfl', 'typevalue_Antibiotics', 'typevalue_Inotropes',
       'urine', 'urine_per_hour',
       'urine_per_kg_hour','gender', 'birthweight',
       'birthlength', 'birthheadcircumference', 'inout_patient_status',
       'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'gestation']

In [5]:
gd = gs[cols_to_use]

In [6]:
def range_finder(x):
    length = x
    fractional = (x/15.0) - math.floor(x/15.0)
    return int(round(fractional*15))

In [7]:
import math
final_df = pd.DataFrame(columns=gd.columns)
for i in gd.uhid.unique():
    x = gd[gd['uhid']==i]
    x = x[range_finder(len(x)):len(x)]
    
    final_df = final_df.append(x,ignore_index=True)
    

In [15]:
train = final_df[:2520]
test = final_df[2520:]

In [16]:
y_train = train['dischargestatus']
X_train = train.drop('dischargestatus',axis=1)
X_train = X_train.drop('uhid',axis=1)
#X_train = X_train.drop('visittime',axis=1)

y_test = test['dischargestatus']
X_test = test.drop('dischargestatus',axis=1)
X_test = X_test.drop('uhid',axis=1)
#X_test = X_test.drop('startdate',axis=1)

In [17]:
#converting the data into a numpy array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

ytrain1 = []
for i in range(0,len(y_train),15):
    #print(i)
    y1 = y_train[i:i+15]
    ytrain1.append(y1[-1])
    
ytest1 = []
for i in range(0,len(y_test),15):
    #print(i)
    y1 = y_test[i:i+15]
    ytest1.append(y1[-1])
    
ytrain1 = np.array(ytrain1)
ytest1 = np.array(ytest1)

Xtrain = np.reshape(X_train, (-1, 15, X_train.shape[1]))
Xtest = np.reshape(X_test, (-1, 15, X_test.shape[1]))

In [18]:
auc_roc_inter = []
from sklearn.metrics import roc_auc_score
val_a = []
train_a = []

In [19]:
for i in range(5):
    #Building the LSTM model
    X = Input(shape=(None, 48), name='X')
    mX = Masking()(X)
    lstm = Bidirectional(LSTM(units=512,activation='tanh',return_sequences=True,recurrent_dropout=0.5,dropout=0.3))
    mX = lstm(mX)
    L = LSTM(units=64,activation='tanh',return_sequences=False)(mX)
    y = Dense(1, activation="sigmoid")(L)
    outputs = [y]
    inputs = [X]
    model = Model(inputs,outputs)
    model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

    
    #fitting the model
    model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
    #history = model.fit(Xtrain, ytrain1, batch_size=60 ,validation_split=0.15,epochs=38,callbacks=[es])
    for i in range(len(model.history.history['val_acc'])):
        val_a.append(model.history.history['val_acc'][i])
        train_a.append(model.history.history['acc'][i])
    #predictions
    y_pred = model.predict(Xtest)
    #y_pred = y_pred.round()
    y_test = np.array(ytest1)
    y_pred = np.array(y_pred)
    y_test = pd.DataFrame(y_test)
    y_test = np.array(y_test)

    def acc(x):
        if x>0.5:
            return 1
        else:
            return 0

    y_model=[]
    for i in y_pred:
        y_model.append(acc(i))
    y_answer=[]
    for j in y_test:
        y_answer.append(acc(j))
    
    auc_roc_inter.append(roc_auc_score(y_answer,y_pred))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 142 samples, validate on 26 samples
Epoch 1/38
142/142 [==============================] - 4s 31ms/step - loss: 0.6522 - acc: 0.5915 - val_loss: 0.5744 - val_acc: 0.5769
Epoch 2/38
142/142 [==============================] - 0s 2ms/step - loss: 0.5127 - acc: 0.6831 - val_loss: 0.6307 - val_acc: 0.6923
Epoch 3/38
142/142 [==============================] - 0s 2ms/step - loss: 0.4351 - acc: 0.7535 - val_loss: 0.3595 - val_acc: 0.8462
Epoch 4/38
142/142 [==============================] - 0s 2ms/step - loss: 0.3329 - acc: 0.9225 - val_loss: 0.2270 - val_acc: 0.8462
Epoch 5/38
142/142 [==============================] - 0s 2ms/step - loss: 0.2700 - acc: 0.9296 - val_loss: 0.5684 - val_acc: 0.6923
Epoch 6/38
142/142 [==============================] - 0s 2ms/step - loss: 0.262

In [20]:
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [21]:
mean_confidence_interval(auc_roc_inter)

(0.7925146198830408, 0.7350217429652577, 0.850007496800824)

In [25]:
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_answer,y_model)

In [33]:
ppv = (cm[0][0])/(cm[0][0]+cm[0][1])
npv = (cm[1][1])/(cm[1][0]+cm[1][1])

In [36]:
print("The PPV and NPV:")
print(round(ppv,2),round(npv,2))

The PPV and NPV:
0.86 0.53
